In [2]:
# from google.colab import drive
# drive.mount('/content/drive')


In [3]:
# file_path = '/content/drive/My Drive/merged_news.csv'


In [4]:
import pandas as pd
# df = pd.read_csv(file_path)
df = pd.read_csv("merged_news.csv")

df.head()


,title,link,published,source,description,source_type,label,scraped_at,fact_check_link,fact_check_verdict,subject
0,Putin announces three-day Russian ceasefire in...,https://www.bbc.com/news/articles/c62j1848509o,"Mon, 28 Apr 2025 13:40:51 GMT",BBC News,"Ukraine calls for an ""immediate"" ceasefire of ...",credible,unknown,2025-04-28T15:46:01.579561,https://factcheck.afp.com/doc.afp.com.42BV3CV,altered picture,unknown
1,"Girl, 14, who stabbed teachers sentenced to 15...",https://www.bbc.com/news/articles/c3v9y544wq6o,"Mon, 28 Apr 2025 13:10:44 GMT",BBC News,Teenager who stabbed two teachers and a pupil ...,credible,unknown,2025-04-28T15:46:01.579715,NaN,NaN,unknown
2,Ultra-processed foods may be linked to early d...,https://www.bbc.com/news/articles/crm30kwvv17o,"Mon, 28 Apr 2025 12:47:47 GMT",BBC News,"UPFs, such as crisps, biscuits, ice cream, hot...",credible,unknown,2025-04-28T15:46:01.579841,NaN,NaN,unknown
3,No extra cash for teacher and NHS staff pay ri...,https://www.bbc.com/news/articles/cpq7324qdj9o,"Mon, 28 Apr 2025 12:49:40 GMT",BBC News,Pay review bodies recommend increases for teac...,credible,unknown,2025-04-28T15:46:01.580020,NaN,NaN,unknown
4,UK could see hottest April weather in seven ye...,https://www.bbc.com/weather/articles/cy489e83j74o,"Mon, 28 Apr 2025 13:28:13 GMT",BBC News,Temperatures are climbing this week with most ...,credible,unknown,2025-04-28T15:46:01.580127,https://science.feedback.org/review/breitbart-...,inaccurate,unknown


In [5]:
print("Shape:", df.shape)  # (rows, columns)
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])


Shape: (45450, 11)
Number of rows: 45450
Number of columns: 11


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Features and labels
df['text'] = df['title'].astype(str) + " " + df['description'].astype(str)

X = df['text']
y = df['label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [7]:
print(df[['text', 'label']].head())


                                                text    label
0  Putin announces three-day Russian ceasefire in...  unknown
1  Girl, 14, who stabbed teachers sentenced to 15...  unknown
2  Ultra-processed foods may be linked to early d...  unknown
3  No extra cash for teacher and NHS staff pay ri...  unknown
4  UK could see hottest April weather in seven ye...  unknown


In [8]:
print(df['label'].value_counts())


label
fake       23608
true       21417
unknown      423
real           2
Name: count, dtype: int64


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Create and train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Predict on the test data
y_pred = nb_model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print results
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")

print(class_report)


Accuracy: 92.39%
Confusion Matrix:
[[4408    0  257    0]
 [   0    0    1    0]
 [ 351    0 3990    0]
 [  38    0   45    0]]
Classification Report:
              precision    recall  f1-score   support

        fake       0.92      0.94      0.93      4665
        real       0.00      0.00      0.00         1
        true       0.93      0.92      0.92      4341
     unknown       0.00      0.00      0.00        83

    accuracy                           0.92      9090
   macro avg       0.46      0.47      0.46      9090
weighted avg       0.92      0.92      0.92      9090



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
